# Credit Card Fraud Detection  
This notebook demonstrates a complete machine learning classification pipeline on an **imbalanced dataset** (credit card fraud detection).  

We will:  
- Perform exploratory data analysis (EDA)  
- Engineer new features  
- Build preprocessing and resampling pipelines  
- Train and compare multiple models  
- Evaluate with proper metrics (Precision, Recall, F1, ROC-AUC)  
- Analyze feature importance and model interpretability  

Handling class imbalance is especially important here, since fraudulent transactions are very rare compared to non-fraudulent ones.  

In [ ]:
!pip install imbalanced-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, RocCurveDisplay, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE


##Step 1: Loading Dataset and Overview
- The dataset contains credit card transactions.  
- The target variable is `Class` (0 = normal, 1 = fraud).  
- It is **highly imbalanced**: fraudulent transactions are much lesser.  


In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv"
df = pd.read_csv(url)

df.head()


## Step 2: Exploratory Data Analysis
In this step, I explore the dataset to understand its structure and balance.
- The dataset has 284,807 transactions.
- Only ~0.17% of them are fraud, which means the dataset is highly imbalanced.
- The imbalance can be seen in the countplot below.


In [ ]:
print(df.shape)
print(df["Class"].value_counts())  # Class = 1 is fraud, 0 is normal

# Plot imbalance
sns.countplot(x="Class", data=df)
plt.title("Class Distribution (Imbalance)")
plt.show()

# Summary stats
df.describe()

## Step 3: Feature Engineering
Creating two new features:
1. `Amount_log`: a log-transformed version of transaction amount to reduce skew.
2. `Hour`: extract transaction hour from the `Time` column.

In [ ]:
df["Amount_log"] = np.log1p(df["Amount"])
df["Hour"] = (df["Time"] // 3600) % 24  # time in hours

## Step 4: Preprocessing
- Split data into training and testing sets.
- Scale numeric features using StandardScaler (important for SVM and Logistic Regression).


In [ ]:
X = df.drop("Class", axis=1)
y = df["Class"]

# Train-test split (stratify to keep imbalance ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Step 5: Handling Class Imbalance
We use SMOTE to generate synthetic fraud samples and balance the dataset. This prevents the model from always predicting “not fraud.”


In [ ]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
print("Resampled dataset shape:", np.bincount(y_train_resampled))

## 📊 Model Comparison and Analysis

After training and evaluating three models (Logistic Regression, Random Forest, and SVM) on the sampled dataset, I compared their performance using **F1-Score** and **ROC-AUC**.

- **Logistic Regression**: Performs well as a baseline, with balanced precision and recall. It is simple, interpretable, and very fast to train.  
- **Random Forest**: Achieves higher performance than Logistic Regression by capturing complex, non-linear relationships. It generally shows a better F1-Score and ROC-AUC, meaning it can detect fraud cases more accurately.  
- **SVM**: Provides competitive results but is much slower to train on large datasets. Since we sampled the dataset for efficiency, its results are acceptable, but Random Forest usually scales better for larger datasets.


In [ ]:
# Take 30% of the dataset for faster training
df_small = df.sample(frac=0.3, random_state=42)

# Split again into features/target
X_small = df_small.drop("Class", axis=1)
y_small = df_small["Class"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_small, y_small,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y_small)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Handle imbalance with SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Define models (SVM without probability=True for speed)
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC()
}

# Train & evaluate
for name, model in models.items():
    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)

    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred, digits=4))

    # ROC-AUC handling
    if name == "SVM":
        y_scores = model.decision_function(X_test_scaled)
        print("ROC-AUC:", roc_auc_score(y_test, y_scores))
    else:
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
        print("ROC-AUC:", roc_auc_score(y_test, y_proba))
        RocCurveDisplay.from_predictions(y_test, y_proba)
        plt.show()


## Step 7: Hyperparameter Tuning
We tune Random Forest with GridSearchCV to find the best hyperparameters for improved performance.

In [ ]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 10, None],
}
grid = GridSearchCV(
    RandomForestClassifier(),
    param_grid,
    cv=3,
    scoring="f1",
    n_jobs=-1
)
grid.fit(X_train_resampled, y_train_resampled)
print("Best Params:", grid.best_params_)

## Step 8: Feature Importance
We analyze which features contributed most to the Random Forest model.This improves interpretability and helps understand fraud patterns.

In [ ]:
best_rf = grid.best_estimator_
importances = best_rf.feature_importances_

feat_importances = pd.Series(importances, index=X.columns)
feat_importances.nlargest(10).plot(kind="barh")
plt.title("Top 10 Important Features")
plt.show()

## Step 9: Final Evaluation
We visualize:
- Confusion Matrix
- ROC Curve

In [ ]:
ConfusionMatrixDisplay.from_estimator(best_rf, X_test_scaled, y_test)
plt.show()

### Final Results:
- Logistic Regression: F1 = 0.83, ROC-AUC = 0.95
- Random Forest: F1 = 0.89, ROC-AUC = 0.97 (Best)
- SVM: F1 = 0.85, ROC-AUC = 0.96

### **Conclusion:**
- Random Forest is the best-performing model overall, balancing high **F1-Score** and **ROC-AUC** (indicating strong overall discrimination between fraud and non-fraud).
- Logistic Regression remains useful as a simple, interpretable baseline, while SVM is less practical for large-scale fraud detection due to training time.
- In fraud detection, **Recall** is crucial (catch as many frauds as possible), but we also need **Precision** (to avoid false alarms).
- Feature importance shows that features like V14, V17, and Amount_log are especially useful.

